### Hw-2: Task 2
### Taslima Akter
### ID: takter

In [22]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

# Imports
import numpy as np
import tensorflow as tf

import os
# os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
# os.environ["CUDA_VISIBLE_DEVICES"]="0"
import tensorflow as tf
import numpy as np
import time
import matplotlib.pyplot as plt
from operator import itemgetter
from sklearn.datasets import fetch_mldata

from keras.models import Model
from keras.layers import Input, LSTM, GRU, Dense, Dropout, Bidirectional
from keras.models import Sequential
from keras.layers import Dense
from keras.utils.np_utils import to_categorical

%matplotlib notebook

In [2]:
config = tf.ConfigProto()
# config.gpu_options.allow_growth = True
# config.gpu_options.per_process_gpu_memory_fraction = 0.33

### Read file names for training data

In [3]:
### File names for Training Data

import glob
import librosa
fname_trn=[]
for filename in glob.glob('/opt/e533/timit-homework/tr/trn*.wav'):
    (fname_trn.append(filename))
fname_trn.sort()
# fname_trn

fname_trs=[]
for filename in glob.glob('/opt/e533/timit-homework/tr/trs*.wav'):
    (fname_trs.append(filename))
fname_trs.sort()
# fname_trs

fname_trx=[]
for filename in glob.glob('/opt/e533/timit-homework/tr/trx*.wav'):
    (fname_trx.append(filename))
fname_trx.sort()
fname_trx[100]


'/opt/e533/timit-homework/tr/trx0100.wav'

### Read file names for validation data

In [4]:
### File names for validation Data

fname_val_n=[]
for filename in glob.glob('/opt/e533/timit-homework/v/vn*.wav'):
    (fname_val_n.append(filename))
fname_val_n.sort()
# print(fname_val_n)

fname_val_s=[]
for filename in glob.glob('/opt/e533/timit-homework/v/vs*.wav'):
    (fname_val_s.append(filename))
fname_val_s.sort()
# fname_trs

fname_val_x=[]
for filename in glob.glob('/opt/e533/timit-homework/v/vx*.wav'):
    (fname_val_x.append(filename))
fname_val_x.sort()


### Read file names for test data

In [5]:
### File names for Test Data

fname_test=[]
for filename in glob.glob('/opt/e533/timit-homework/te/tex*.wav'):
    (fname_test.append(filename))
fname_test.sort()


### Write data into txt file

In [8]:
### Function For writing file
import librosa
def write_file(file_name, fname_list):
    ### Writing training data S

    with open(file_name, 'wb') as fs:
        for i in range(len(fname_list)):
            sn, sr=librosa.load(fname_list[i], sr=None)
            Sn=librosa.stft(sn, n_fft=1024, hop_length=512)
            mag_Sn=np.abs(Sn)
    #         trn_arr=np.concatenate((trn_arr, mag_Sn), axis=1)
            np.savetxt(fs, mag_Sn, fmt='%.5f')
            fs.write(b'\n')
    fs.close()            

    

### Writing complex data X into file 

In [9]:
### Function For writing just X into file
import librosa
def write_file_X(file_name, fname_list):
    ### Writing training data S

    with open(file_name, 'wb') as fs:
        for i in range(len(fname_list)):
            sn, sr=librosa.load(fname_list[i], sr=None)
            Sn=librosa.stft(sn, n_fft=1024, hop_length=512)
#             mag_Sn=np.abs(Sn)
    #         trn_arr=np.concatenate((trn_arr, mag_Sn), axis=1)
            np.savetxt(fs, Sn, fmt='%.5f')
            fs.write(b'\n')
    fs.close()            

    

In [41]:
### Writing training data S
Train_complx_X=[]

for i in range(len(fname_trx)):
    print(i)
    
    sn, sr=librosa.load(fname_trx[i], sr=None)
    Sn=librosa.stft(sn, n_fft=1024, hop_length=512)
    Train_complx_X.append(np.array(Sn))


#             mag_Sn=np.abs(Sn)
#         trn_arr=np.concatenate((trn_arr, mag_Sn), axis=1)
   


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [45]:
### Writing training data S
val_complx_X=[]

for i in range(len(fname_val_x)):
    print(i)
    
    sn, sr=librosa.load(fname_val_x[i], sr=None)
    Sn=librosa.stft(sn, n_fft=1024, hop_length=512)
    val_complx_X.append(np.array(Sn))


#             mag_Sn=np.abs(Sn)
#         trn_arr=np.concatenate((trn_arr, mag_Sn), axis=1)
   


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [88]:
### Writing training data S
val_complx_S=[]

for i in range(len(fname_val_s)):
    print(i)
    
    sn, sr=librosa.load(fname_val_s[i], sr=None)
    Sn=librosa.stft(sn, n_fft=1024, hop_length=512)
    val_complx_S.append(np.array(Sn))


#             mag_Sn=np.abs(Sn)
#         trn_arr=np.concatenate((trn_arr, mag_Sn), axis=1)
   


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [124]:
### Writing training data S
test_complx_S=[]

for i in range(len(fname_test)):
    print(i)
    
    sn, sr=librosa.load(fname_test[i], sr=None)
    Sn=librosa.stft(sn, n_fft=1024, hop_length=512)
    test_complx_S.append(np.array(Sn))


#             mag_Sn=np.abs(Sn)
#         trn_arr=np.concatenate((trn_arr, mag_Sn), axis=1)
   


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

### Calling functions to write data

In [ ]:
write_file("train_s.txt", fname_trs)
write_file("train_n.txt", fname_trn)
write_file("train_x.txt", fname_trx)

In [6]:
### Write validation files

write_file("validation_s.txt", fname_val_s)
write_file("validation_n.txt", fname_val_n)
write_file("validation_x.txt", fname_val_x)


In [8]:
write_file_X("train_x_tr.txt", fname_trx)
write_file_X("validation_x_tr.txt", fname_trx)


In [8]:
write_file("test_data.txt", fname_test)

In [9]:
### Writing training data N
import librosa

count=0
total_train_s=[]
with open('train_n.txt', 'wb') as fn:
    for i in range(len(fname_trn)):
        sn, sr=librosa.load(fname_trn[i], sr=None)
        Sn=librosa.stft(sn, n_fft=1024, hop_length=512)
        mag_Sn=np.abs(Sn)
#         trn_arr=np.concatenate((trn_arr, mag_Sn), axis=1)
        np.savetxt(fn, mag_Sn, fmt='%.5f')
        fn.write(b'\n')
fn.close()            

In [10]:
### Writing training data S

with open('train_s.txt', 'wb') as fs:
    for i in range(len(fname_trs)):
        sn, sr=librosa.load(fname_trs[i], sr=None)
        Sn=librosa.stft(sn, n_fft=1024, hop_length=512)
        mag_Sn=np.abs(Sn)
#         trn_arr=np.concatenate((trn_arr, mag_Sn), axis=1)
        np.savetxt(fs, mag_Sn, fmt='%.5f')
        fs.write(b'\n')
fs.close()            


In [11]:
### Writing training file X

with open('train_x.txt', 'wb') as fs:
    for i in range(len(fname_trx)):
        sn, sr=librosa.load(fname_trx[i], sr=None)
        Sn=librosa.stft(sn, n_fft=1024, hop_length=512)
        mag_Sn=np.abs(Sn)
#         trn_arr=np.concatenate((trn_arr, mag_Sn), axis=1)
        np.savetxt(fs, mag_Sn, fmt='%.5f')
        fs.write(b'\n')
fs.close()            


### Read data from files

In [10]:
### Function for Reading file

def read_file(file_name):
    with open(file_name) as f:
        lines=f.readlines()
        print(len(lines))
        sentence_full=[]
        count = 0
        sentence=[]
        for line in lines:

            if count < 513:
                if count ==0:
                    sentence=np.array(np.fromstring(line, dtype=complex_, sep=' '), ndmin=2)
                    count+=1
                else:
                    myarray = np.array(np.fromstring(line, dtype=complex_, sep=' '), ndmin=2)
                    sentence=np.concatenate((sentence, myarray), axis=0)
                    count+=1
            else:
                sentence_full.append(sentence) 
                count=0
                sentence=[]
        return sentence_full


### Read training data

In [50]:
data_train_n = read_file("train_n.txt")
data_train_s = read_file("train_s.txt")
data_train_x = read_file("train_x.txt")

616800
616800
616800


### Read validation data

In [51]:
data_val_n = read_file("validation_n.txt")
data_val_s = read_file("validation_s.txt")
data_val_x = read_file("validation_x.txt")

616800
616800
616800


### Read complex X from training and validation data

In [23]:
data_train_xtr = read_file("train_x_tr.txt")
data_val_xtr = read_file("validation_x_tr.txt")

616800
616800


In [29]:
len(data_val_xtr)
data_val_xtr[0].shape

(513, 1)

In [54]:
data_test = read_file("test_data.txt")

205600


### Calculating M for training and validation data

In [55]:
### Calculating M:
data_train_M=[]
data_val_M=[]
for i in range(len(data_train_s)):
    data_train_M.append(1*(data_train_s[i]>data_train_n[i]))
    data_val_M.append(1*(data_val_s[i]>data_val_n[i]))
    
    

In [64]:
data_train_M[2].shape

(513, 65)

### Creating Batch  

In [97]:
def next_batchXSCmplx(X_, S_, X_cmplx, S_cmplx):
    
    batch_x = None
    batch_s = None
    batch_x_cmplx = None
    batch_s_cmplx = None
    
    for e,(x, s, x_cmplx, s_cmplx) in enumerate(zip(X_, S_, X_cmplx, S_cmplx)): 
        batch_x = np.array(x.T) if batch_x is None else np.concatenate( (batch_x,x.T), axis=0)
        batch_s = np.array(s.T) if batch_s is None else np.concatenate( (batch_s,s.T), axis=0)
        batch_x_cmplx = np.array(x_cmplx.T) if batch_x_cmplx is None else np.concatenate( (batch_x_cmplx,x_cmplx.T), axis=0)
        batch_s_cmplx = np.array(s_cmplx.T) if batch_s_cmplx is None else np.concatenate( (batch_s_cmplx,s_cmplx.T), axis=0)
 
        if e>0 and (e+1)%10==0:
            temp_x, batch_x = batch_x, None
            temp_s, batch_s = batch_s, None
            temp_x_cmplx, batch_x_cmplx = batch_x_cmplx, None
            temp_s_cmplx, batch_s_cmplx = batch_s_cmplx, None
            
            temp_x = temp_x.reshape((-1,Max_RNN,513))
            temp_s = temp_s.reshape((-1,Max_RNN,513))
            temp_x_cmplx = temp_x_cmplx.reshape((-1,Max_RNN,513))
#             temp_s_cmplx = temp_s_cmplx.reshape((-1,Max_RNN,513))

            yield temp_x, temp_s, temp_x_cmplx, temp_s_cmplx

In [ ]:
def next_batchX(X_):
    
    batch_x = None
    
    for e,x in enumerate(X_): 
        batch_x = np.array(x.T) if batch_x is None else np.concatenate( (batch_x,x.T), axis=0)
        
        if e>0 and (e+1)%10==0:
            temp_x, batch_x = batch_x, None
            
            temp_x = temp_x.reshape((-1,Max_RNN,513))

            yield temp_x

In [56]:
def next_batch(X_,Y_):
    
    batch_x, batch_y = None, None
    
    for e,(x,y) in enumerate(zip(X_,Y_)):
#         print(e)
        
        batch_x = np.array(x.T) if batch_x is None else np.concatenate( (batch_x,x.T), axis=0)
        batch_y = np.array(y.T) if batch_y is None else np.concatenate( (batch_y,y.T), axis=0)
        
#         print('batch_x',batch_x.shape)
#         print('batch_y',batch_y.shape)
        
        if e>0 and (e+1)%10==0:
            temp_x, batch_x = batch_x, None
            temp_y, batch_y = batch_y, None
            
            temp_x = temp_x.reshape((-1,Max_RNN,513))
            temp_y = temp_y.reshape((-1,Max_RNN,513))

#             print('temp_x',temp_x.shape)
#             print('temp_y',temp_y.shape)
        
            yield temp_x,temp_y


### RNN implementation

In [112]:
Max_RNN=5
model = Sequential()

model.add(Bidirectional(GRU(Max_RNN, return_sequences=True), input_shape=(Max_RNN,513)))
model.add(Dropout(0.2))
model.add(Bidirectional(GRU(Max_RNN, return_sequences=True)))
# model.add(GRU(output_dim = 513, input_length = 5, input_dim = 513, return_sequences=True))

# model.add(Activation('relu'))
# model.add(TimeDistributed(Dense(513, activation='sigmoid')))
model.add(Dense(513, activation='sigmoid'))

model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
print(model.summary())

for e in range(10):
    for (b_x,b_y), (v_x,v_y) in zip(next_batch(data_train_x, data_train_M), next_batch(data_val_x, data_val_M)):
        model.fit(b_x, b_y, validation_data=(v_x,v_y), shuffle=True, batch_size=100)

#     model.fit( , epochs=20, steps_per_epoch=700, validation_data=next_batch_mb(DATA_val_x, DATA_val_x,10), validation_steps=700)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional_9 (Bidirection (None, 5, 10)             20760     
_________________________________________________________________
dropout_5 (Dropout)          (None, 5, 10)             0         
_________________________________________________________________
bidirectional_10 (Bidirectio (None, 5, 10)             640       
_________________________________________________________________
dense_5 (Dense)              (None, 5, 513)            5643      
Total params: 27,043
Trainable params: 27,043
Non-trainable params: 0
_________________________________________________________________
None
Train on 130 samples, validate on 130 samples
Epoch 1/1
130/130 [==============================] - 3s 22ms/step - loss: 0.6931 - acc: 0.5045 - val_loss: 0.6929 - val_acc: 0.5286
Train on 260 samples, validate on 260 samples
Epoch 1/1
260/260 [======================

Train on 210 samples, validate on 210 samples
Epoch 1/1
210/210 [==============================] - 0s 740us/step - loss: 0.6770 - acc: 0.5557 - val_loss: 0.6715 - val_acc: 0.5570
Train on 158 samples, validate on 158 samples
Epoch 1/1
158/158 [==============================] - 0s 668us/step - loss: 0.6558 - acc: 0.6350 - val_loss: 0.6667 - val_acc: 0.5627
Train on 198 samples, validate on 198 samples
Epoch 1/1
198/198 [==============================] - 0s 601us/step - loss: 0.6563 - acc: 0.6127 - val_loss: 0.6635 - val_acc: 0.5837
Train on 206 samples, validate on 206 samples
Epoch 1/1
206/206 [==============================] - 0s 744us/step - loss: 0.6847 - acc: 0.5008 - val_loss: 0.6723 - val_acc: 0.5474
Train on 180 samples, validate on 180 samples
Epoch 1/1
180/180 [==============================] - 0s 646us/step - loss: 0.6611 - acc: 0.5799 - val_loss: 0.6523 - val_acc: 0.6077
Train on 214 samples, validate on 214 samples
Epoch 1/1
214/214 [==============================] - 0s 747

Train on 228 samples, validate on 228 samples
Epoch 1/1
228/228 [==============================] - 0s 687us/step - loss: 0.6811 - acc: 0.5430 - val_loss: 0.6607 - val_acc: 0.6203
Train on 202 samples, validate on 202 samples
Epoch 1/1
202/202 [==============================] - 0s 757us/step - loss: 0.6468 - acc: 0.6444 - val_loss: 0.6447 - val_acc: 0.6357
Train on 180 samples, validate on 180 samples
Epoch 1/1
180/180 [==============================] - 0s 598us/step - loss: 0.6570 - acc: 0.5971 - val_loss: 0.6479 - val_acc: 0.6475
Train on 242 samples, validate on 242 samples
Epoch 1/1
242/242 [==============================] - 0s 734us/step - loss: 0.7030 - acc: 0.5378 - val_loss: 0.6988 - val_acc: 0.5683
Train on 144 samples, validate on 144 samples
Epoch 1/1
144/144 [==============================] - 0s 695us/step - loss: 0.6511 - acc: 0.6339 - val_loss: 0.6502 - val_acc: 0.6327
Train on 356 samples, validate on 356 samples
Epoch 1/1
356/356 [==============================] - 0s 617

Train on 154 samples, validate on 154 samples
Epoch 1/1
154/154 [==============================] - 0s 687us/step - loss: 0.6252 - acc: 0.6312 - val_loss: 0.6186 - val_acc: 0.6789
Train on 262 samples, validate on 262 samples
Epoch 1/1
262/262 [==============================] - 0s 662us/step - loss: 0.6648 - acc: 0.5967 - val_loss: 0.6442 - val_acc: 0.6556
Train on 156 samples, validate on 156 samples
Epoch 1/1
156/156 [==============================] - 0s 678us/step - loss: 0.6021 - acc: 0.7044 - val_loss: 0.5761 - val_acc: 0.7391
Train on 158 samples, validate on 158 samples
Epoch 1/1
158/158 [==============================] - 0s 670us/step - loss: 0.6434 - acc: 0.6392 - val_loss: 0.6698 - val_acc: 0.6402
Train on 188 samples, validate on 188 samples
Epoch 1/1
188/188 [==============================] - 0s 617us/step - loss: 0.6164 - acc: 0.6907 - val_loss: 0.6093 - val_acc: 0.6994
Train on 216 samples, validate on 216 samples
Epoch 1/1
216/216 [==============================] - 0s 746

Train on 118 samples, validate on 118 samples
Epoch 1/1
118/118 [==============================] - 0s 836us/step - loss: 0.6212 - acc: 0.6720 - val_loss: 0.6259 - val_acc: 0.6637
Train on 232 samples, validate on 232 samples
Epoch 1/1
232/232 [==============================] - 0s 679us/step - loss: 0.6144 - acc: 0.6724 - val_loss: 0.6221 - val_acc: 0.6824
Train on 150 samples, validate on 150 samples
Epoch 1/1
150/150 [==============================] - 0s 681us/step - loss: 0.5600 - acc: 0.7488 - val_loss: 0.5813 - val_acc: 0.7312
Train on 144 samples, validate on 144 samples
Epoch 1/1
144/144 [==============================] - 0s 702us/step - loss: 0.6350 - acc: 0.6543 - val_loss: 0.6100 - val_acc: 0.6975
Train on 156 samples, validate on 156 samples
Epoch 1/1
156/156 [==============================] - 0s 654us/step - loss: 0.5830 - acc: 0.7098 - val_loss: 0.5644 - val_acc: 0.7335
Train on 212 samples, validate on 212 samples
Epoch 1/1
212/212 [==============================] - 0s 729

...

Train on 186 samples, validate on 186 samples
Epoch 1/1
186/186 [==============================] - 0s 637us/step - loss: 0.5670 - acc: 0.7025 - val_loss: 0.5366 - val_acc: 0.7304
Train on 180 samples, validate on 180 samples
Epoch 1/1
180/180 [==============================] - 0s 611us/step - loss: 0.5373 - acc: 0.7390 - val_loss: 0.5293 - val_acc: 0.7442
Train on 222 samples, validate on 222 samples
Epoch 1/1
222/222 [==============================] - 0s 695us/step - loss: 0.5504 - acc: 0.7268 - val_loss: 0.5500 - val_acc: 0.7340
Train on 296 samples, validate on 296 samples
Epoch 1/1
296/296 [==============================] - 0s 559us/step - loss: 0.5492 - acc: 0.7225 - val_loss: 0.5326 - val_acc: 0.7408
Train on 230 samples, validate on 230 samples
Epoch 1/1
230/230 [==============================] - 0s 674us/step - loss: 0.5238 - acc: 0.7446 - val_loss: 0.5245 - val_acc: 0.7472
Train on 190 samples, validate on 190 samples
Epoch 1/1
190/190 [==============================] - 0s 573

Train on 234 samples, validate on 234 samples
Epoch 1/1
234/234 [==============================] - 0s 662us/step - loss: 0.5059 - acc: 0.7553 - val_loss: 0.4646 - val_acc: 0.7836
Train on 184 samples, validate on 184 samples
Epoch 1/1
184/184 [==============================] - 0s 593us/step - loss: 0.5068 - acc: 0.7641 - val_loss: 0.5140 - val_acc: 0.7578
Train on 238 samples, validate on 238 samples
Epoch 1/1
238/238 [==============================] - 0s 631us/step - loss: 0.5583 - acc: 0.7163 - val_loss: 0.4989 - val_acc: 0.7617
Train on 340 samples, validate on 340 samples
Epoch 1/1
340/340 [==============================] - 0s 602us/step - loss: 0.5318 - acc: 0.7404 - val_loss: 0.5127 - val_acc: 0.7525
Train on 220 samples, validate on 220 samples
Epoch 1/1
220/220 [==============================] - 0s 696us/step - loss: 0.5179 - acc: 0.7580 - val_loss: 0.5268 - val_acc: 0.7481
Train on 260 samples, validate on 260 samples
Epoch 1/1
260/260 [==============================] - 0s 596

Train on 176 samples, validate on 176 samples
Epoch 1/1
176/176 [==============================] - 0s 574us/step - loss: 0.5305 - acc: 0.7397 - val_loss: 0.5448 - val_acc: 0.7225
Train on 212 samples, validate on 212 samples
Epoch 1/1
212/212 [==============================] - 0s 705us/step - loss: 0.5240 - acc: 0.7410 - val_loss: 0.5245 - val_acc: 0.7434
Train on 330 samples, validate on 330 samples
Epoch 1/1
330/330 [==============================] - 0s 618us/step - loss: 0.5701 - acc: 0.7044 - val_loss: 0.5674 - val_acc: 0.7141
Train on 180 samples, validate on 180 samples
Epoch 1/1
180/180 [==============================] - 0s 568us/step - loss: 0.5650 - acc: 0.7008 - val_loss: 0.4848 - val_acc: 0.7670
Train on 256 samples, validate on 256 samples
Epoch 1/1
256/256 [==============================] - 0s 603us/step - loss: 0.5391 - acc: 0.7285 - val_loss: 0.5046 - val_acc: 0.7657
Train on 264 samples, validate on 264 samples
Epoch 1/1
264/264 [==============================] - 0s 573

Train on 194 samples, validate on 194 samples
Epoch 1/1
194/194 [==============================] - 0s 580us/step - loss: 0.5302 - acc: 0.7273 - val_loss: 0.5028 - val_acc: 0.7578
Train on 164 samples, validate on 164 samples
Epoch 1/1
164/164 [==============================] - 0s 635us/step - loss: 0.5842 - acc: 0.6929 - val_loss: 0.5566 - val_acc: 0.7091
Train on 346 samples, validate on 346 samples
Epoch 1/1
346/346 [==============================] - 0s 595us/step - loss: 0.5464 - acc: 0.7225 - val_loss: 0.5242 - val_acc: 0.7425
Train on 172 samples, validate on 172 samples
Epoch 1/1
172/172 [==============================] - 0s 607us/step - loss: 0.5337 - acc: 0.7414 - val_loss: 0.5180 - val_acc: 0.7614
Train on 286 samples, validate on 286 samples
Epoch 1/1
286/286 [==============================] - 0s 540us/step - loss: 0.4864 - acc: 0.7783 - val_loss: 0.4855 - val_acc: 0.7771
Train on 186 samples, validate on 186 samples
Epoch 1/1
186/186 [==============================] - 0s 560

Train on 154 samples, validate on 154 samples
Epoch 1/1
154/154 [==============================] - 0s 646us/step - loss: 0.5435 - acc: 0.7349 - val_loss: 0.5428 - val_acc: 0.7442
Train on 166 samples, validate on 166 samples
Epoch 1/1
166/166 [==============================] - 0s 635us/step - loss: 0.5127 - acc: 0.7572 - val_loss: 0.4928 - val_acc: 0.7678
Train on 224 samples, validate on 224 samples
Epoch 1/1
224/224 [==============================] - 0s 670us/step - loss: 0.5044 - acc: 0.7632 - val_loss: 0.4809 - val_acc: 0.7796
Train on 294 samples, validate on 294 samples
Epoch 1/1
294/294 [==============================] - 0s 600us/step - loss: 0.5376 - acc: 0.7346 - val_loss: 0.5179 - val_acc: 0.7482
Train on 150 samples, validate on 150 samples
Epoch 1/1
150/150 [==============================] - 0s 663us/step - loss: 0.4972 - acc: 0.7534 - val_loss: 0.4928 - val_acc: 0.7700
Train on 196 samples, validate on 196 samples
Epoch 1/1
196/196 [==============================] - 0s 569

### Calculating Accuracy

In [120]:
scores = []
for v_x,v_y in next_batch(data_val_x, data_val_M):
    scores.append( model.evaluate(v_x, v_y,verbose=0)[1] )
scores=np.mean(scores)
print("Accuracy: ",  scores*100)

Accuracy:  75.0041942597735


### Calculating SNR

In [ ]:
sum_s = 0.0
sum_s_diff = 0.0

for v_s,v_x,v_x_cmplx,v_s_cmplx in next_batchXSCmplx(data_val_s,data_val_x,val_complx_X,val_complx_S):
    
#     print(v_s.shape)
#     print(v_x.shape)
#     print(v_x_cmplx.shape)
#     print('v_s_cmplx',v_s_cmplx.shape)
    
    mask = model.predict(v_x)
    S_hat = (mask) * v_x_cmplx
    S_hat = S_hat.reshape(-1,513).T
    S = v_s_cmplx.T
#     S=S.reshape(-1,513)
    
#     print('S.shape',S.shape)
#     print('S_hat.shape',S_hat.shape)

    S_org = librosa.istft(S, hop_length=512)
    S_pred = librosa.istft(S_hat, hop_length=512)

    sum_s += np.sum(S_org*S_org)
    sum_s_diff += np.sum((S_org-S_pred)*(S_org-S_pred))
    
acc = sum_s/ sum_s_diff
print(acc)

10*np.log10(acc)

### Write into audio file

In [122]:
def write_audio(file_name):
    for i in range(len(file_name)):
        audio_fname=file_name[i].replace("/opt/e533/timit-homework/te/", "").replace(".wav","")
        print(audio_fname)
        sn, sr=librosa.load(file_name[i], sr=None)
        Sn=librosa.stft(sn, n_fft=1024, hop_length=512)
        mag_Sn=np.abs(Sn)
        print(mag_Sn.shape)
#         mag_Sn=mag_Sn.reshape(-1, 5, 513)
        Stest_hat=model.predict(mag_Sn.reshape(-1, 5, 513))
        Stest_hat=Stest_hat.reshape(-1,513)
        S_hat=(Sn/mag_Sn)*Stest_hat.T

        S_time=librosa.istft(S_hat, hop_length=512)
        audio_fname=audio_fname + "_recons.wav"
        print(audio_fname)
        librosa.output.write_wav(audio_fname, S_time, sr)
        
        
    

In [150]:
def write_audio():
    mags = None
    cmplxs = None
    
    for e, file_x in enumerate(fname_test[:10]):
        print(e)
        sn, sr = librosa.load(file_x, sr=None)
        Sn = librosa.stft(sn, n_fft=1024, hop_length=512)
        mag_Sn=np.abs(Sn)
        
        mags = np.array(mag_Sn.T) if mags is None else np.concatenate( (mags,mag_Sn.T), axis=0)
        cmplxs = np.array(Sn.T) if cmplxs is None else np.concatenate( (cmplxs,Sn.T), axis=0)
        
        if e>0 and (e+1)%10==0:
            temp, mags = mags, None
            
            temp = temp.reshape((-1,Max_RNN,513))
            mask = model.predict(temp)
            
            mask=mask.reshape(-1,513)
            S_hat = (mask) * cmplxs
            S_hat = S_hat.T
            
            lenght_w = S_hat.shape[1]//10
            for clip in range(10):
                start_w = clip*lenght_w
                end_w = (clip+1)*lenght_w
                
                wav = S_hat[:,start_w:end_w].T
                S_time=librosa.istft(wav, hop_length=512)
#                 fname = PATH_directory+PATH_denoise+ e + "_redoise.wav"
#                 audio_fname=file_x.replace("/opt/e533/timit-homework/te/", "").replace(".wav","")
                audio_fname=str(clip) + "_recons.wav"
                print(audio_fname)
                librosa.output.write_wav(audio_fname, S_time, sr)
                

In [151]:
write_audio()

0
1
2
3
4
5
6
7
8
9
0_recons.wav
1_recons.wav
2_recons.wav
3_recons.wav
4_recons.wav
5_recons.wav
6_recons.wav
7_recons.wav
8_recons.wav
9_recons.wav


In [18]:
import librosa

sn, sr=librosa.load(fname_trn[0], sr=None)
Sn=librosa.stft(sn, n_fft=1024, hop_length=512)
trn_arr=np.abs(Sn)
print(trn_arr.shape)
sx, sr=librosa.load(fname_trx[0], sr=None)
Sx=librosa.stft(sx, n_fft=1024, hop_length=512)
trx_arr=np.abs(Sx)

ss, sr=librosa.load(fname_trs[0], sr=None)
Ss=librosa.stft(ss, n_fft=1024, hop_length=512)
trs_arr=np.abs(Ss)
for i in range(1,1200):

    sn, sr=librosa.load(fname_trn[i], sr=None)
    Sn=librosa.stft(sn, n_fft=1024, hop_length=512)
    mag_Sn=np.abs(Sn)
    trn_arr=np.concatenate((trn_arr, mag_Sn), axis=1)
#     print(Sn.shape)

    sx, sr=librosa.load(fname_trx[i], sr=None)
    Sx=librosa.stft(sx, n_fft=1024, hop_length=512)
    mag_Sx=np.abs(Sx)
    trx_arr=np.concatenate((trx_arr, mag_Sx), axis=1)
    
    ss, sr=librosa.load(fname_trs[i], sr=None)
    Ss=librosa.stft(ss, n_fft=1024, hop_length=512)
    mag_Ss=np.abs(Ss)
    trs_arr=np.concatenate((trs_arr, mag_Ss), axis=1)
# print(Sn.shape)

# sn, sr=librosa.load(fname_trn[129], sr=None)
# Sn=librosa.stft(sn, n_fft=1024, hop_length=512)

# print(Sn.shape)

(513, 65)


In [20]:
DATA_train_M = 1*(trs_arr>trn_arr)
DATA_train_M.shape

(513, 118550)

In [40]:
trn_arr = trn_arr.reshape( (513,10,-1))
trx_arr = trx_arr.reshape( (513,10,-1))
trs_arr = trs_arr.reshape( (513,10,-1))
DATA_train_M = DATA_train_M.reshape( (513,10,-1))

trn_arr.shape, trx_arr.shape, trs_arr.shape, DATA_train_M.shape


((513, 10, 11855), (513, 10, 11855), (513, 10, 11855), (513, 10, 11855))

In [14]:
import librosa

sn, sr=librosa.load(fname_val_n[0], sr=None)
Sn=librosa.stft(sn, n_fft=1024, hop_length=512)
val_n_arr=np.abs(Sn)

# print(trn_arr.shape)
sx, sr=librosa.load(fname_val_x[0], sr=None)
Sx=librosa.stft(sx, n_fft=1024, hop_length=512)
val_x_arr=np.abs(Sx)

ss, sr=librosa.load(fname_val_s[0], sr=None)
Ss=librosa.stft(ss, n_fft=1024, hop_length=512)
val_s_arr=np.abs(Ss)
for i in range(1,len(fname_val_n)):

    sn, sr=librosa.load(fname_val_n[i], sr=None)
    Sn=librosa.stft(sn, n_fft=1024, hop_length=512)
    mag_Sn=np.abs(Sn)
    val_n_arr=np.concatenate((val_n_arr, mag_Sn), axis=1)
#     print(Sn.shape)

    sx, sr=librosa.load(fname_val_x[i], sr=None)
    Sx=librosa.stft(sx, n_fft=1024, hop_length=512)
    mag_Sx=np.abs(Sx)
    val_x_arr=np.concatenate((val_x_arr, mag_Sx), axis=1)
    
    ss, sr=librosa.load(fname_val_s[i], sr=None)
    Ss=librosa.stft(ss, n_fft=1024, hop_length=512)
    mag_Ss=np.abs(Ss)
    val_s_arr=np.concatenate((val_s_arr, mag_Ss), axis=1)
# print(Sn.shape)

# sn, sr=librosa.load(fname_trn[129], sr=None)
# Sn=librosa.stft(sn, n_fft=1024, hop_length=512)

# print(Sn.shape)

In [15]:
DATA_val_M = 1*(val_s_arr>val_n_arr)
DATA_val_M.shape

(513, 118550)

In [41]:
val_n_arr = val_n_arr.reshape( (513,10,-1))
val_x_arr = val_x_arr.reshape( (513,10,-1))
val_s_arr = val_s_arr.reshape( (513,10,-1))
DATA_val_M = DATA_val_M.reshape( (513,10,-1))

val_n_arr.shape, val_x_arr.shape, val_s_arr.shape, DATA_val_M.shape


((513, 10, 11855), (513, 10, 11855), (513, 10, 11855), (513, 10, 11855))

In [8]:
sn, sr=librosa.load(fname_test[0], sr=None)
Sn=librosa.stft(sn, n_fft=1024, hop_length=512)
test_arr=np.abs(Sn)

# print(trn_arr.shape)
# sx, sr=librosa.load(fname_val_x[0], sr=None)
# Sx=librosa.stft(sx, n_fft=1024, hop_length=512)
# val_x_arr=np.abs(Sx)

# ss, sr=librosa.load(fname_val_s[0], sr=None)
# Ss=librosa.stft(ss, n_fft=1024, hop_length=512)
# val_s_arr=np.abs(Ss)
for i in range(1,len(fname_test)):

    sn, sr=librosa.load(fname_test[i], sr=None)
    Sn=librosa.stft(sn, n_fft=1024, hop_length=512)
    mag_Sn=np.abs(Sn)
    test_arr=np.concatenate((test_arr, mag_Sn), axis=1)


In [42]:
test_arr.shape
test_arr = test_arr.reshape( (513,10,-1))
test_arr.shape

(513, 10, 4415)

In [43]:
np.save('DATA_train_s.npy', trs_arr)
np.save('DATA_train_n.npy', trn_arr)
np.save('DATA_train_x.npy', trx_arr)
np.save('DATA_train_M.npy', DATA_train_M)

np.save('DATA_val_s.npy', val_s_arr)
np.save('DATA_val_n.npy', val_n_arr)
np.save('DATA_val_x.npy', val_x_arr)
np.save('DATA_val_M.npy', DATA_val_M)

np.save('DATA_test_x.npy', test_arr)

In [44]:
DATA_train_s_f = np.load('DATA_train_s.npy')
DATA_train_n_f = np.load('DATA_train_n.npy')
DATA_train_x_f = np.load('DATA_train_x.npy')
DATA_train_M_f = np.load('DATA_train_M.npy')

DATA_val_s_f = np.load('DATA_val_s.npy')
DATA_val_n_f = np.load('DATA_val_n.npy')
DATA_val_x_f = np.load('DATA_val_x.npy')
DATA_val_M_f = np.load('DATA_val_M.npy')

DATA_test_x_f = np.load('DATA_test_x.npy')

In [111]:
# DATA_val_M_f[:10, :3, :10]

In [45]:
tr_categorical_labels = to_categorical(DATA_train_M_f.T, num_classes=2)
print(tr_categorical_labels.shape)

v_categorical_labels = to_categorical(DATA_val_M_f.T, num_classes=2)
print(v_categorical_labels.shape)

(11855, 10, 513, 2)
(11855, 10, 513, 2)


In [54]:
Max_RNN = 10
# create the model

model = Sequential()

model.add(LSTM(Max_RNN, return_sequences=True, input_shape=(Max_RNN,513)))
model.add(Dropout(0.2))

model.add(Dense(513, activation='sigmoid'))
          
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])
print(model.summary())

# model.fit(DATA_train_x_f.T, tr_categorical_labels, validation_data=(DATA_val_x_f.T,v_categorical_labels), shuffle=True, nb_epoch=100, batch_size=10)
# # Final evaluation of the model
# scores = model.evaluate(DATA_val_x_f.T, v_categorical_labels, verbose=0)



model.fit(DATA_train_x_f.T, DATA_train_M_f.T, validation_data=(DATA_val_x_f.T,DATA_val_M_f.T), shuffle=True, nb_epoch=25, batch_size=10)
# Final evaluation of the model
scores = model.evaluate(DATA_val_x_f.T, DATA_val_M_f.T, verbose=0)



print("Accuracy: %.2f%%" % (scores[1]*100))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_5 (LSTM)                (None, 10, 10)            20960     
_________________________________________________________________
dropout_5 (Dropout)          (None, 10, 10)            0         
_________________________________________________________________
dense_5 (Dense)              (None, 10, 513)           5643      
Total params: 26,603
Trainable params: 26,603
Non-trainable params: 0
_________________________________________________________________
None


/opt/Anaconda3-5.0.1/envs/e533/lib/python3.6/site-packages/keras/models.py:942: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Train on 11855 samples, validate on 11855 samples
Epoch 1/25
11855/11855 [==============================] - 20s 2ms/step - loss: 0.2133 - acc: 0.0017 - val_loss: 0.1920 - val_acc: 0.0018
Epoch 2/25
11855/11855 [==============================] - 20s 2ms/step - loss: 0.1944 - acc: 0.0026 - val_loss: 0.1836 - val_acc: 0.0022
Epoch 3/25
11855/11855 [==============================] - 19s 2ms/step - loss: 0.1891 - acc: 0.0033 - val_loss: 0.1782 - val_acc: 0.0022
Epoch 4/25
11855/11855 [==============================] - 20s 2ms/step - loss: 0.1859 - acc: 0.0049 - val_loss: 0.1751 - val_acc: 0.0035
Epoch 5/25
11855/11855 [==============================] - 19s 2ms/step - loss: 0.1838 - acc: 0.0076 - val_loss: 0.1735 - val_acc: 0.0071
Epoch 6/25
11855/11855 [==============================] - 19s 2ms/step - loss: 0.1824 - acc: 0.0124 - val_loss: 0.1718 - val_acc: 0.0091
Epoch 7/25
11855/11855 [==============================] - 19s 2ms/step - loss: 0.1812 - acc: 0.0162 - val_loss: 0.1712 - val_acc

In [55]:
import librosa

sx, sr=librosa.load(fname_trx[0], sr=None)
Sx=librosa.stft(sx, n_fft=1024, hop_length=512)
X_tr=Sx

for i in range(1,len(fname_trx)):

    sx, sr=librosa.load(fname_trx[i], sr=None)
    Sx=librosa.stft(sx, n_fft=1024, hop_length=512)
#     mag_Sx=np.abs(Sx)
    X_tr=np.concatenate((X_tr, Sx), axis=1)
    


In [ ]:
import librosa

sx, sr=librosa.load(fname_val_x[0], sr=None)
Sx=librosa.stft(sx, n_fft=1024, hop_length=512)
X_val=Sx

for i in range(1,len(fname_val_x)):

    sx, sr=librosa.load(fname_val_x[i], sr=None)
    Sx=librosa.stft(sx, n_fft=1024, hop_length=512)
#     mag_Sx=np.abs(Sx)
    X_val=np.concatenate((X_val, Sx), axis=1)
    


In [48]:
labels = model.predict(DATA_val_x_f.T)
labels.shape

(11855, 10, 513)

In [49]:
S_hat = (labels.T) * DATA_val_x_f

In [50]:
S_org = (DATA_val_s_f).flatten()
S_pred = S_hat.flatten()

In [51]:
acc = np.sum(S_org*S_org)/ np.sum((S_org-S_pred)*(S_org-S_pred))
print(acc)

10*np.log10(acc)

4.074747


6.101006865501404